In [188]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import syft as sy
import copy
import numpy as np
import time
import binarytree

import importlib
importlib.import_module('FLDataset')
from FLDataset import load_dataset, getActualImgs
from utils import averageModels, normalModels

In [189]:
class Arguments():
    def __init__(self):
        self.images = 60000
        self.clients = 20
        self.rounds = 10
        self.epochs = 5
        self.local_batches = 64
        self.lr = 0.01
        self.C = 1.0
        self.drop_rate = 0.0
        self.mu = 0.5
        self.momentum = 0.6
        self.dampening = 1.0
        self.torch_seed = 0
        self.log_interval = 100
        self.iid = 'iid'
        self.split_size = int(self.images / self.clients)
        self.samples = self.split_size / self.images 
        self.use_cuda = False
        self.save_model = False

args = Arguments()

use_cuda = args.use_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [190]:
writer = open('prox.csv', 'a')

In [191]:
hook = sy.TorchHook(torch)
clients = []

for i in range(args.clients):
    clients.append({'hook': sy.VirtualWorker(hook, id="client{}".format(i+1))})

In [192]:
# # Download MNIST manually using 'wget' then uncompress the file
# !wget www.di.ens.fr/~lelarge/MNIST.tar.gz
# !tar -zxvf MNIST.tar.gz

In [193]:
global_train, global_test, train_group, test_group = load_dataset(args.clients, args.iid)

In [194]:
for inx, client in enumerate(clients):
    trainset_ind_list = list(train_group[inx])
    client['trainset'] = getActualImgs(global_train, trainset_ind_list, args.local_batches)
    client['testset'] = getActualImgs(global_test, list(test_group[inx]), args.local_batches)
#     client['samples'] = len(trainset_ind_list) / args.images
    client['samples'] = len(trainset_ind_list) / (len(trainset_ind_list) * args.clients)

In [195]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
global_test_dataset = datasets.FashionMNIST('./', train=False, download=True, transform=transform)
global_test_loader = DataLoader(global_test_dataset, batch_size=args.local_batches, shuffle=True)

In [196]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [197]:
def ClientUpdate(args, device, client):
    client['model'].train()
#     client['model'].send(client['hook'])
    
    for epoch in range(1, args.epochs + 1):
        for batch_idx, (data, target) in enumerate(client['trainset']):
#             data = data.send(client['hook'])
#             target = target.send(client['hook'])
            
            data, target = data.to(device), target.to(device)
            client['optim'].zero_grad()
            output = client['model'](data)
            loss = F.nll_loss(output, target)
            loss.backward()
            client['optim'].step()
#             client['optim'].step(global_model)
    
            
            if batch_idx % args.log_interval == 0:
#                 loss = loss.get() 
                print('Model {} Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    client['hook'].id,
                    epoch, batch_idx * args.local_batches, len(client['trainset']) * args.local_batches, 
                    100. * batch_idx / len(client['trainset']), loss))
                
#     client['model'].get() 

In [198]:
def test(args, model, device, test_loader, name):
    model.eval()   
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss for {} model: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        name, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    writer.write('{},'.format(100. * correct / len(test_loader.dataset)))

In [199]:
class FedProxOptim(optim.Optimizer):
    def __init__(self, params, lr=args.lr, mu=args.mu, momentum=args.momentum, dampening=args.dampening):
        defaults = dict(lr=lr, mu=mu, momentum=momentum, dampening=dampening)
        super(FedProxOptim, self).__init__(params, defaults)
    
    def step(self, global_model=None, closure = None):
        loss = None
        if closure is not None:
            loss = closure()
        for group in self.param_groups:
            lr, mu, momentum, dampening = group['lr'], group['mu'], group['momentum'], group['dampening']
            for p in zip(group['params'], list(global_model.parameters())):
                if p[0].grad is None:
                    continue
                d_p = p[0].grad.data # local model grads
                if momentum != 0:
                    param_state = self.state[p[0]]
                    if 'momentum_buffer' not in param_state:
                        buf = param_state['momentum_buffer'] = d_p.clone()
                    else:
                        buf = param_state['momentum_buffer']
                        buf.mul_(momentum).add_(1 - dampening, d_p)
                p[0].data.sub_(group['lr'], (d_p + mu * (p[0].data.clone() - p[1].data.clone())))
                
        return loss

In [200]:
def Aggregate(all_models, sample_client):
    all_dicts = {}
    for client in all_models:
        all_dicts[client] = all_models[client].state_dict()
    output_dict = copy.deepcopy(all_dicts[sample_client])
    for key in all_dicts[sample_client]:
        for val in all_dicts:
            if val == sample_client:
                continue
            output_dict[key] += all_dicts[val][key]
#         output_dict[key] /= float(len(all_dicts))
        output_dict[key] /= float(len(all_models))
    output_model = Net().to(device)
    output_model.load_state_dict(output_dict)
    return output_model

In [201]:
def BtreeAvg(root):
    global global_model
    temp_model = Aggregate({'root':root.value[1], 'global':global_model}, 'global')
    
    if (root.left == None and root.right == None): 
        return temp_model
    left = None
    right = None
    if root.left:
        left = BtreeAvg(root.left)
    if root.right:
        right = BtreeAvg(root.right)

    if left and right:
        return Aggregate({'root':root.value[1], 'left':left, 'right':right}, 'left')
    if left:
        return Aggregate({'root':root.value[1], 'left':left}, 'left')
    if right:
        return Aggregate({'root':root.value[1],'right':right}, 'right')
 

In [202]:
def BtreeAvgSecond(root, final):
    temp_model = Aggregate({'root':root.value[1], 'global':final}, 'global')
    
    if (root.left == None and root.right == None): 
        return temp_model
    left = None
    right = None
    if root.left:
        left = BtreeAvgSecond(root.left, temp_model)
    if root.right:
        right = BtreeAvgSecond(root.right, temp_model)

    if left and right:
        return Aggregate({'root':root.value[1], 'left':left, 'right':right}, 'left')
    if left:
        return Aggregate({'root':root.value[1], 'left':left}, 'left')
    if right:
        return Aggregate({'root':root.value[1],'right':right}, 'right')
 

In [203]:
torch.manual_seed(args.torch_seed)
global_model = Net()
# torch.manual_seed(args.torch_seed)
# normal_avg = Net()

for client in clients:
    torch.manual_seed(args.torch_seed)
    client['model'] = Net().to(device)
    client['optim'] = optim.SGD(client['model'].parameters(), lr=args.lr, momentum=args.momentum)
#     client['optim'] = optim.Adam(client['model'].parameters(), lr=args.lr)
#     client['optim'] = FedProxOptim(client['model'].parameters(), lr=args.lr, mu=args.mu, momentum=args.momentum, dampening=args.dampening)

tree_List = []
for inds, i in enumerate(clients):
    temp = [None] * 3
    temp[0] = i
    temp[1] = clients[inds]['model']
    temp[2] = clients[inds]['hook'].id
    tree_List.append(temp)

root = None
root = binarytree.insertLevelOrder(tree_List, root, 0, len(tree_List))
# root.printTree()

start_time = time.time()

for fed_round in range(args.rounds):
    
#     uncomment if you want a randome fraction for C every round
#     args.C = float(format(np.random.random(), '.1f'))
    
    # number of selected clients
    m = int(max(args.C * args.clients, 1))

    # Selected devices
    np.random.seed(fed_round)
    selected_clients_inds = np.random.choice(range(len(clients)), m, replace=False)
    selected_clients = [clients[i] for i in selected_clients_inds]
    
    # Active devices
    np.random.seed(fed_round)
    active_clients_inds = np.random.choice(selected_clients_inds, int((1-args.drop_rate) * m), replace=False)
    active_clients = [clients[i] for i in active_clients_inds]
    print(len(active_clients))
    
    # Training 
    for client in active_clients:
        ClientUpdate(args, device, client)
        
#     global_model = averageModels(global_model, active_clients)
#     test(args, global_model, device, global_test_loader, 'Global')
#     normal_avg = normalModels(normal_avg, active_clients)
#     test(args, normal_avg, device, global_test_loader, 'Global')
    
    
    # Begin the Btree
#     final_model = BtreeAvg(root)
    final_model = BtreeAvgSecond(root, global_model)
#     glo_model = copy.deepcopy(final_model)
    global_model.load_state_dict(final_model.state_dict())
#     test(args, global_model, device, global_test_loader, 'Global')
#     break
    
#     # Testing 
#     for client in active_clients:
#         test(args, client['model'], device, client['testset'], client['hook'].id)
    
#     # Averaging 
#     global_model = averageModels(global_model, active_clients)
    
    # Testing the average model
    test(args, global_model, device, global_test_loader, 'Global')
            
    # Share the global model with the clients
    for client in clients:
        client['model'].load_state_dict(global_model.state_dict())

writer.write('\n')
end_time = time.time() 
hours, rem = divmod(end_time-start_time, 3600)
minutes, seconds = divmod(rem, 60)
writer.write('{:0>2}:{:0>2}:{:05.2f}'.format(int(hours),int(minutes),seconds))
writer.write('\n\n')
writer.close()

if (args.save_model):
    torch.save(global_model.state_dict(), "FedAvg.pt")

20
Model client16 Train Epoch: 1 [0/3008 (0%)]	Loss: 2.302624
Model client16 Train Epoch: 2 [0/3008 (0%)]	Loss: 1.199924
Model client16 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.853815
Model client16 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.910680
Model client16 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.859631
Model client2 Train Epoch: 1 [0/3008 (0%)]	Loss: 2.317857
Model client2 Train Epoch: 2 [0/3008 (0%)]	Loss: 1.532781
Model client2 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.883139
Model client2 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.811264
Model client2 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.783275
Model client13 Train Epoch: 1 [0/3008 (0%)]	Loss: 2.316322
Model client13 Train Epoch: 2 [0/3008 (0%)]	Loss: 1.331101
Model client13 Train Epoch: 3 [0/3008 (0%)]	Loss: 1.024799
Model client13 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.759412
Model client13 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.571817
Model client5 Train Epoch: 1 [0/3008 (0%)]	Loss: 2.312681
Model client5 Train Epoch: 2 [0/3008 (0%)]	Loss: 1.399488
M

Model client13 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.677285
Model client18 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.655902
Model client18 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.761058
Model client18 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.649599
Model client18 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.700969
Model client18 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.635892
Model client17 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.725843
Model client17 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.716482
Model client17 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.495388
Model client17 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.441098
Model client17 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.578625
Model client19 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.544413
Model client19 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.670693
Model client19 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.543349
Model client19 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.415511
Model client19 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.475004
Model client4 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.65117

Model client4 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.582209
Model client4 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.586491
Model client9 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.664979
Model client9 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.576197
Model client9 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.512803
Model client9 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.410810
Model client9 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.352269
Model client3 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.517768
Model client3 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.528971
Model client3 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.612129
Model client3 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.577748
Model client3 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.517465
Model client7 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.626029
Model client7 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.645890
Model client7 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.501940
Model client7 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.634714
Model client7 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.606571
Model client2 

Model client1 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.484944
Model client1 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.526883
Model client1 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.445925
Model client1 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.359636
Model client3 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.310303
Model client3 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.401936
Model client3 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.534241
Model client3 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.376145
Model client3 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.365247
Model client14 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.502927
Model client14 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.569050
Model client14 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.306604
Model client14 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.516280
Model client14 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.618421
Model client20 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.376593
Model client20 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.468405
Model client20 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.383352
Model 

Model client13 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.550743
Model client13 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.307196
Model client13 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.531351
Model client13 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.330016
Model client13 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.322386
Model client14 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.384449
Model client14 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.358014
Model client14 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.406147
Model client14 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.484177
Model client14 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.361175
Model client19 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.406158
Model client19 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.279528
Model client19 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.268188
Model client19 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.515113
Model client19 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.364222
Model client3 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.427964
Model client3 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.465247

Model client12 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.175642
Model client13 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.396481
Model client13 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.428969
Model client13 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.379235
Model client13 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.279283
Model client13 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.223124

Test set: Average loss for Global model: 0.4112, Accuracy: 8518/10000 (85%)

20
Model client18 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.552164
Model client18 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.375753
Model client18 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.423039
Model client18 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.414822
Model client18 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.229836
Model client4 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.360651
Model client4 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.396657
Model client4 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.319262
Model client4 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.315209
Model client4 Train Epoch: 5 [0/3008 (

Model client15 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.357453
Model client15 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.460152
Model client15 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.236042
Model client7 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.372033
Model client7 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.433908
Model client7 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.263883
Model client7 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.403501
Model client7 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.394938
Model client20 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.395276
Model client20 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.334540
Model client20 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.342634
Model client20 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.307408
Model client20 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.227305
Model client12 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.295928
Model client12 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.363217
Model client12 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.349511
Model client12 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.226936
Mo

Model client15 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.335278
Model client15 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.385437
Model client15 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.409462
Model client15 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.186760
Model client1 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.423091
Model client1 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.412869
Model client1 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.252966
Model client1 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.283668
Model client1 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.076924
Model client2 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.490849
Model client2 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.193199
Model client2 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.413351
Model client2 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.297355
Model client2 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.436514
Model client18 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.323286
Model client18 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.384851
Model client18 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.323994
Model c

In [204]:
clients[0]['optim']

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0.6
    nesterov: False
    weight_decay: 0
)